## Necessary Imports

In [209]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.engine import URL

## Define the Connection

In [210]:
conn_str = (
        r"Driver=ODBC Driver 17 for SQL Server;"
        r"Server=DS01;"
        r"Database=Sandbox_Intern_LD;"
        r"Trusted_Connection=yes;"
    )

conn_url = URL.create("mssql+pyodbc", query={"odbc_connect": conn_str})

engine = create_engine(conn_url)

## Read the Table

In [211]:
# query = 'SELECT * FROM LeanHogs_re20240617 ORDER BY MarketTradeDate DESC;'
query = 'SELECT * FROM LeanHogs_re20240617 AS t1 WHERE Volume = (SELECT Max(Volume) FROM LeanHogs_re20240617 AS t2 WHERE t1.MarketTradeDate = t2.MarketTradeDate) ORDER BY MarketTradeDate DESC;'
df = pd.read_sql_query(query, con=engine)
df.head()

,MarketTradeDate,FuturesContractSpecId,OpenPx,HighPx,LowPx,SettlementPx,Volume,OpenInterest,DailyOK,ContractDate,ExpiryDate,LastTradingUTCDateTime,FirstNoticeDate,SpotContractInd
0,2024-06-14,49,-1.000000e+09,-1.000000e+09,-1.000000e+09,84.900,0,-1000000000,True,2025-05-01,2025-05-14 17:00:00,2025-05-14 17:00:00,None,0
1,2024-06-14,49,-1.000000e+09,-1.000000e+09,-1.000000e+09,72.225,0,-1000000000,True,2025-12-01,2025-12-12 18:00:00,2025-12-12 18:00:00,None,0
2,2024-06-14,49,-1.000000e+09,-1.000000e+09,-1.000000e+09,77.925,0,-1000000000,True,2025-10-01,2025-10-14 17:00:00,2025-10-14 17:00:00,None,0
3,2024-06-14,49,9.145000e+01,9.147500e+01,9.115000e+01,91.175,0,-1000000000,True,2024-06-01,2024-06-14 17:00:00,2024-06-14 17:00:00,None,1
4,2024-06-14,49,9.150000e+01,9.190000e+01,9.045000e+01,91.875,0,-1000000000,True,2025-06-01,2025-06-13 17:00:00,2025-06-13 17:00:00,None,0


## Generate Timeseries

In [212]:
df['ContractDate'] = pd.to_datetime(df['ContractDate'], errors='coerce')
df['MarketTradeDate'] = pd.to_datetime(df['MarketTradeDate'], errors='coerce')
# df = df[df['ContractDate'].dt.month == 7]
df.head()

,MarketTradeDate,FuturesContractSpecId,OpenPx,HighPx,LowPx,SettlementPx,Volume,OpenInterest,DailyOK,ContractDate,ExpiryDate,LastTradingUTCDateTime,FirstNoticeDate,SpotContractInd
0,2024-06-14,49,-1.000000e+09,-1.000000e+09,-1.000000e+09,84.900,0,-1000000000,True,2025-05-01,2025-05-14 17:00:00,2025-05-14 17:00:00,None,0
1,2024-06-14,49,-1.000000e+09,-1.000000e+09,-1.000000e+09,72.225,0,-1000000000,True,2025-12-01,2025-12-12 18:00:00,2025-12-12 18:00:00,None,0
2,2024-06-14,49,-1.000000e+09,-1.000000e+09,-1.000000e+09,77.925,0,-1000000000,True,2025-10-01,2025-10-14 17:00:00,2025-10-14 17:00:00,None,0
3,2024-06-14,49,9.145000e+01,9.147500e+01,9.115000e+01,91.175,0,-1000000000,True,2024-06-01,2024-06-14 17:00:00,2024-06-14 17:00:00,None,1
4,2024-06-14,49,9.150000e+01,9.190000e+01,9.045000e+01,91.875,0,-1000000000,True,2025-06-01,2025-06-13 17:00:00,2025-06-13 17:00:00,None,0


In [213]:
# for i, row in df.iterrows():
#     # print(row['ContractDate'].year, row['MarketTradeDate'].year)
#     if row['ContractDate'].year - 1 == row['MarketTradeDate'].year:
#         if row['MarketTradeDate'].month >= 7:
#             continue
#     if row['ContractDate'].year == row['MarketTradeDate'].year:
#         if row['MarketTradeDate'].month < 7:
#             continue
#     df.drop(i, inplace=True)

# df.head()

In [214]:
# for i, row in df.iterrows():
#     if i % 1000 == 0:
#         print(f"{i}/{len(df)}")
#     contract = row['ContractDate']
#     market = row['MarketTradeDate']
#     if contract.month == 1:
#         if market.month == 12 and market.year == contract.year - 1:
#             continue
#     if contract.month - 1 == market.month:
#         if contract.year == market.year:
#             continue
#     df.drop(i, inplace=True)

# df.head()

In [215]:
# df = df[df['SpotContractInd'] == 1]
# df.head()

## Restrict Columns and Save as CSV

In [216]:
df.reset_index(inplace=True)
df.drop(columns=['FuturesContractSpecId', 'DailyOK', 'ExpiryDate', 'LastTradingUTCDateTime', 'FirstNoticeDate', 'SpotContractInd', 'index'], inplace=True)
df.head()

,MarketTradeDate,OpenPx,HighPx,LowPx,SettlementPx,Volume,OpenInterest,ContractDate
0,2024-06-14,-1.000000e+09,-1.000000e+09,-1.000000e+09,84.900,0,-1000000000,2025-05-01
1,2024-06-14,-1.000000e+09,-1.000000e+09,-1.000000e+09,72.225,0,-1000000000,2025-12-01
2,2024-06-14,-1.000000e+09,-1.000000e+09,-1.000000e+09,77.925,0,-1000000000,2025-10-01
3,2024-06-14,9.145000e+01,9.147500e+01,9.115000e+01,91.175,0,-1000000000,2024-06-01
4,2024-06-14,9.150000e+01,9.190000e+01,9.045000e+01,91.875,0,-1000000000,2025-06-01


In [217]:
df.to_csv("../Data/SQL_LHF.csv")

## 